In [1]:
import requests
from bs4 import BeautifulSoup
# Get the main url books data 
url='http://books.toscrape.com/catalogue/category/books_1/index.html'
result=requests.get(url)
soup=BeautifulSoup(result.text,'html.parser')

In [2]:
# Get every category url and append them in a list to scrape
cat_link=soup.find("ul",{"class":"nav nav-list"}).find("ul").find_all("a")
lst=[]
for i in cat_link :
    lst.append('http://books.toscrape.com/catalogue/category'+i['href'].replace("..",""))

In [3]:
# Create dictionary and put the name of the movie as a key and the values of each key(book name) are price, category and rating
dct={}
# Loop for each category url and scrape it 
for e in lst :
    result=requests.get(e)
    soup=BeautifulSoup(result.text,'html.parser')
    # get the numbe of pages for each category
    no_pages=soup.find_all("ul",{"class":"pager"})
    # check if the number of page for the category is one then scrape it and add each book to the dictionary
    if len(no_pages)==0 :
        cat=soup.find("h1").text
        book_name=soup.find_all("h3")
        book_price=soup.find_all("p",{"class":"price_color"})
        book_rate=soup.find_all('article',{'class':'product_pod'})
        for x,y,z in zip(book_name,book_price,book_rate) :
            dct[x.find("a")["title"]]=[cat,float(y.text[2:]),z.p.attrs['class'][1]]
    # If the number of the pages for the category is bigger than one then loop for each page in it and scrape it and then add it to the dictionary
    else :
        no_pages=int(no_pages[0].find("li",{"class":"current"}).text.split("\n")[2].strip().split(" ")[3])
        for num in range(1,no_pages+1,1) :
            page_url=e.replace("index.html","page-")+str(num)+".html"
            result=requests.get(page_url)
            soup=BeautifulSoup(result.text,'html.parser')
            cat=soup.find("h1").text
            book_name=soup.find_all("h3")
            book_price=soup.find_all("p",{"class":"price_color"})
            book_rate=soup.find_all('article',{'class':'product_pod'})
            for x,y in zip(book_name,book_price) :
                dct[x.find("a")["title"]]=[cat,float(y.text[2:]),z.p.attrs['class'][1]]
# Get each category name and append then into dictionary
cat_list=[]
for i in list(dct.values()) :
    cat_list.append(i[0])
cat_list=list(dict.fromkeys(cat_list))

In [4]:
# Update the list that exist in each dictionary having price , rating and category to have category code instead
for eachbook_cat in dct.values() :
    eachbook_cat[0]=cat_list.index(eachbook_cat[0])+1
# Update the list that exist in each dictionary having price , rating and category to have bookname also
for book_nam,book_detail in dct.items() :
    book_detail=book_detail.insert(0,book_nam)
book_detail_sql=list(dct.values())

In [5]:
# create database and its tables and its columns 
import mysql.connector
conn=mysql.connector.connect(host="localhost",user="root",password="241998")
cursor=conn.cursor()
query="""CREATE DATABASE Books ;
USE books;
CREATE TABLE Categories (
Categories VARCHAR(40),
Book_Category_code INT AUTO_INCREMENT NOT NULL,
PRIMARY KEY (Book_Category_code));
CREATE TABLE Books_Details (
Book_Name VARCHAR(255),
Book_Category_code INT(10),
Book_Price FLOAT (20),
Book_Rating VARCHAR(20),
FOREIGN KEY (Book_Category_code) REFERENCES Categories(Book_Category_code)
);
"""
cursor.execute(query)

In [6]:
# insert our data into database 
for book in cat_list :
    conn=mysql.connector.connect(host="localhost",user="root",password="241998",database="Books")
    cursor=conn.cursor()
    query="""insert into Categories (Categories) values (%s) ; commit"""
    data=(book,)
    cursor.execute(query,data)
for each_book in book_detail_sql :
    conn=mysql.connector.connect(host="localhost",user="root",password="241998",database="Books")
    cursor=conn.cursor()
    query= """insert into Books_Details(Book_Name,Book_Category_code,Book_Price,Book_Rating) values (%s,%s,%s,%s);commit;"""
    data=(each_book[0],each_book[1],each_book[2],each_book[3])
    cursor.execute(query,data)

## Q1 SQL query to get books that has rate >=3 and has 'Mr' in its name.

In [7]:
conn=mysql.connector.connect(host="localhost",user="root",password="241998",database="Books")
cursor=conn.cursor()
query="""SELECT Book_Name
FROM books_details
WHERE (Book_Rating = "Three" OR
Book_Rating = "Four" OR
Book_Rating = "Five") AND 
Book_Name Like "%mr%";"""
cursor.execute(query)
result=cursor.fetchall()
print(result)

[('Mrs. Houdini',)]


## Q2 SQL query to get books that has rate >=3 or price > 20

In [8]:
conn=mysql.connector.connect(host="localhost",user="root",password="241998",database="Books")
cursor=conn.cursor()
query="""SELECT Book_Name
FROM books_details
WHERE (Book_Rating = "Three" OR
Book_Rating = "Four" OR
Book_Rating = "Five") AND 
Book_Price > 20;"""
cursor.execute(query)
result=cursor.fetchall()
print(result)

[('Full Moon over Noahâ\x80\x99s Ark: An Odyssey to Mount Ararat and Beyond',), ('See America: A Celebration of Our National Parks & Treasured Sites',), ('Under the Tuscan Sun',), ('A Year in Provence (Provence #1)',), ('Neither Here nor There: Travels in Europe',), ('1,000 Places to See Before You Die',), ('Sharp Objects',), ('The Past Never Ends',), ('The Murder of Roger Ackroyd (Hercule Poirot #4)',), ('The Last Mile (Amos Decker #2)',), ('A Time of Torment (Charlie Parker #14)',), ('Poisonous (Max Revere Novels #3)',), ('Murder at the 42nd Street Library (Raymond Ambler #1)',), ('Most Wanted',), ('Boar Island (Anna Pigeon #19)',), ('The Widow',), ('What Happened on Beale Street (Secrets of the South Mysteries #2)',), ("The Bachelor Girl's Guide to Murder (Herringford and Watts Mysteries #1)",), ('Delivering the Truth (Quaker Midwife Mystery #1)',), ('The Mysterious Affair at Styles (Hercule Poirot #1)',), ('In the Woods (Dublin Murder Squad #1)',), ('The Silkworm (Cormoran Strike #

## Q3 SQL query to get books that has rate not 3

In [9]:
conn=mysql.connector.connect(host="localhost",user="root",password="241998",database="Books")
cursor=conn.cursor()
query="""SELECT Book_Name
FROM books_details
WHERE (Book_Rating != "Three")"""
cursor.execute(query)
result=cursor.fetchall()
print(result)

[("It's Only the Himalayas",), ('Full Moon over Noahâ\x80\x99s Ark: An Odyssey to Mount Ararat and Beyond',), ('Vagabonding: An Uncommon Guide to the Art of Long-Term World Travel',), ('A Summer In Europe',), ('The Great Railway Bazaar',), ('A Year in Provence (Provence #1)',), ('The Road to Little Dribbling: Adventures of an American in Britain (Notes From a Small Island #2)',), ('1,000 Places to See Before You Die',), ('Sharp Objects',), ('In a Dark, Dark Wood',), ('The Past Never Ends',), ('A Murder in Time',), ('The Murder of Roger Ackroyd (Hercule Poirot #4)',), ('The Last Mile (Amos Decker #2)',), ('That Darkness (Gardiner and Renner #1)',), ('Tastes Like Fear (DI Marnie Rome #3)',), ('A Time of Torment (Charlie Parker #14)',), ('A Study in Scarlet (Sherlock Holmes #1)',), ('Poisonous (Max Revere Novels #3)',), ('Murder at the 42nd Street Library (Raymond Ambler #1)',), ('Most Wanted',), ('Hide Away (Eve Duncan #20)',), ('Boar Island (Anna Pigeon #19)',), ('The Widow',), ('Playin

## Q4 sql query to get books that has price between 10 and 40 and has rate 3 or 4 or 1.

In [10]:
conn=mysql.connector.connect(host="localhost",user="root",password="241998",database="Books")
cursor=conn.cursor()
query="""SELECT Book_Name
FROM books_details
WHERE (Book_Rating = "Three" OR
Book_Rating = "Four" OR
Book_Rating = "One") AND 
Book_Price BETWEEN 10 AND 40;"""
cursor.execute(query)
result=cursor.fetchall()
print(result)

[('Under the Tuscan Sun',), ('The Great Railway Bazaar',), ('The Road to Little Dribbling: Adventures of an American in Britain (Notes From a Small Island #2)',), ('Neither Here nor There: Travels in Europe',), ('The Secret Garden',), ('The Metamorphosis',), ('Little Women (Little Women #1)',), ('Gone with the Wind',), ('Wuthering Heights',), ('The Complete Stories and Poems (The Works of Edgar Allan Poe [Cameo Edition])',), ('Sense and Sensibility',), ('The Stranger',), ('Critique of Pure Reason',), ('Run, Spot, Run: The Ethics of Keeping Pets',), ('The Nicomachean Ethics',), ('Chase Me (Paris Nights #2)',), ('Black Dust',), ('Her Backup Boyfriend (The Sorensen Family #1)',), ('First and First (Five Boroughs #3)',), ('Fifty Shades Darker (Fifty Shades #2)',), ('The Wedding Dress',), ('Something More Than This',), ('Doing It Over (Most Likely To #1)',), ("The Wedding Pact (The O'Malleys #2)",), ('Hold Your Breath (Search and Rescue #1)',), ("A Gentleman's Position (Society of Gentlemen

## Q5 SQL query to get the top 5 most expensive books.

In [11]:
conn=mysql.connector.connect(host="localhost",user="root",password="241998",database="Books")
cursor=conn.cursor()
query="""SELECT Book_Name
FROM books_details
ORDER BY Book_Price DESC
LIMIT 5;"""
cursor.execute(query)
result=cursor.fetchall()
print(result)

[('The Perfect Play (Play by Play #1)',), ('Last One Home (New Beginnings #1)',), ('Civilization and Its Discontents',), ('The Barefoot Contessa Cookbook',), ('The Diary of a Young Girl',)]


## Q6 SQL query to get the 3rd 10 books in the books table order first by rate desc and then by price asc.

In [12]:
conn=mysql.connector.connect(host="localhost",user="root",password="241998",database="Books")
cursor=conn.cursor()
query="""SELECT Book_Name
FROM books_details
ORDER BY Book_Rating DESC,Book_Price ASC
LIMIT 10 OFFSET 20;"""
cursor.execute(query)
result=cursor.fetchall()
print(result)

[('Shoe Dog: A Memoir by the Creator of NIKE',), ('Mexican Today: New and Rediscovered Recipes for Contemporary Kitchens',), ("Surely You're Joking, Mr. Feynman!: Adventures of a Curious Character",), ('Meditations',), ('Soft Apocalypse',), ('Barefoot Contessa Back to Basics',), ('How to Be a Domestic Goddess: Baking and the Art of Comfort Cooking',), ('Like Never Before (Walker Family #2)',), ('The Help Yourself Cookbook for Kids: 60 Easy Plant-Based Recipes Kids Can Make to Stay Healthy and Save the Earth',), ('The Picture of Dorian Gray',)]


 ## Q7 sql query to add a new category in categories table and add 5 books to this category in books table with title, rate and price.

In [13]:
conn=mysql.connector.connect(host="localhost",user="root",password="241998",database="Books")
cursor=conn.cursor()
query= """insert into Categories (Categories) values (%s) ; commit ;"""
data=("Funny",)
cursor.execute(query,data)

In [14]:
conn=mysql.connector.connect(host="localhost",user="root",password="241998",database="Books")
cursor=conn.cursor()
query= """insert into Books_Details(Book_Name,Book_Category_code,Book_Price,Book_Rating) values 
     ("Is Every Thing Hanging Out With Me ? by Mindy Kaling" , 51 , 30 , "Three"),
     ("Me Talk Preety One Day By David Sedaris", 51 , 40 , "Four"),
     ("A dirty Job by Christopher Moore", 51 , 50 , "Five"),
     ("The Ultimate Hitchhikers's Guide to the Galaxy By Douglas Adams",51, 45 , "Two"),
     ("Lets's Pretend This Never Happened by Jenny Lawson", 51 , 10 ,"One");commit;"""
cursor.execute(query)

## Q8 SQL query to update book's rate to 3 that thier price < 20£.

In [15]:
conn=mysql.connector.connect(host="localhost",user="root",password="241998",database="Books")
cursor=conn.cursor()
query="""UPDATE Books_Details
SET Book_Rating = 3
WHERE Book_Price < 20;commit;"""
cursor.execute(query)

## Q9 SQL query to delete all books that have price > 50£ and has rate <= 2.

In [16]:
conn=mysql.connector.connect(host="localhost",user="root",password="241998",database="Books")
cursor=conn.cursor()
query="""DELETE FROM Books_Details
WHERE (Book_Rating = "Two" OR
Book_Rating = "One") And
(Book_Price >50) ;commit;"""
cursor.execute(query)

## Q10 SQL query to count the number of books that have 'Secret' in thier names and price between 10£ and 25£.

In [17]:
conn=mysql.connector.connect(host="localhost",user="root",password="241998",database="Books")
cursor=conn.cursor()
query="""SELECT COUNT(Book_Name)
FROM books_details
WHERE Book_Name LIKE "%Secret%" AND 
Book_Price BETWEEN 10 AND 25;"""
cursor.execute(query)
result=cursor.fetchall()
print(result)

[(5,)]


## Q11 SQL query to get the minimum & maximum price for all the books that have rate 5.

In [18]:
conn=mysql.connector.connect(host="localhost",user="root",password="241998",database="Books")
cursor=conn.cursor()
query="""SELECT MIN(Book_Price),Max(Book_Price)
FROM books_details
WHERE Book_Rating = "Five";"""
cursor.execute(query)
result=cursor.fetchall()
print(result)

[(20.02, 59.48)]


## Q12 SQL query to calculate the avg price for all the books that have rate 5.

In [19]:
conn=mysql.connector.connect(host="localhost",user="root",password="241998",database="Books")
cursor=conn.cursor()
query="""SELECT AVG(Book_Price)
FROM books_details
WHERE Book_Rating = "Five";"""
cursor.execute(query)
result=cursor.fetchall()
print(result)

[(40.538549235447704,)]


## Q13 SQL query to sum all book's price that have rate 2 and price between 10 and 40

In [20]:
conn=mysql.connector.connect(host="localhost",user="root",password="241998",database="Books")
cursor=conn.cursor()
query="""SELECT SUM(Book_Price)
FROM books_details
WHERE Book_Rating = "Two"
AND Book_Price BetWEEN 10 AND 40;"""
cursor.execute(query)
result=cursor.fetchall()
print(result)

[(1254.329990386963,)]


## Q14 SQL query to join both books & categories table into one new table containing book_name & category_name & book_rate and book_price.

In [21]:
conn=mysql.connector.connect(host="localhost",user="root",password="241998",database="Books")
cursor=conn.cursor()
query="""SELECT Book_Name,Categories,Book_Price,book_Rating
FROM Books_Details INNER JOIN Categories 
ON Books_Details.Book_Category_Code=Categories.Book_Category_code;"""
cursor.execute(query)
result=cursor.fetchall()
print(result)

[("It's Only the Himalayas", 'Travel', 45.17, 'Two'), ('Full Moon over Noahâ\x80\x99s Ark: An Odyssey to Mount Ararat and Beyond', 'Travel', 49.43, 'Four'), ('See America: A Celebration of Our National Parks & Treasured Sites', 'Travel', 48.87, 'Three'), ('Vagabonding: An Uncommon Guide to the Art of Long-Term World Travel', 'Travel', 36.94, 'Two'), ('Under the Tuscan Sun', 'Travel', 37.33, 'Three'), ('A Summer In Europe', 'Travel', 44.34, 'Two'), ('The Great Railway Bazaar', 'Travel', 30.54, 'One'), ('A Year in Provence (Provence #1)', 'Travel', 56.88, 'Four'), ('The Road to Little Dribbling: Adventures of an American in Britain (Notes From a Small Island #2)', 'Travel', 23.21, 'One'), ('Neither Here nor There: Travels in Europe', 'Travel', 38.95, 'Three'), ('1,000 Places to See Before You Die', 'Travel', 26.08, 'Five'), ('Sharp Objects', 'Mystery', 47.82, 'Five'), ('In a Dark, Dark Wood', 'Mystery', 19.63, '3'), ('The Past Never Ends', 'Mystery', 56.5, 'Five'), ('A Murder in Time', '

## Q15 SQL query to calculate how many books each rate has and have price between 20£ and 30£.

In [22]:
conn=mysql.connector.connect(host="localhost",user="root",password="241998",database="Books")
cursor=conn.cursor()
query="""SELECT COUNT(Book_Name)
FROM Books_Details 
WHERE Book_Price BETWEEN 20 AND 30
GROUP BY Book_Rating ;"""
cursor.execute(query)
result=cursor.fetchall()
print(result)

[(53,), (48,), (55,), (14,), (38,)]


## Q16 SQL query to calculate how many books each category has having count > 10.

In [23]:
conn=mysql.connector.connect(host="localhost",user="root",password="241998",database="Books")
cursor=conn.cursor()
query="""SELECT COUNT(Book_Name)
FROM Books_Details 
GROUP BY Book_Rating 
HAVING COUNT(Book_Name) >10;"""
cursor.execute(query)
result=cursor.fetchall()
print(result)

[(58,), (223,), (157,), (122,), (193,), (197,)]


## Q17 sql query to get all books with category_name='Music' using subquery.

In [24]:
conn=mysql.connector.connect(host="localhost",user="root",password="241998",database="Books")
cursor=conn.cursor()
query="""SELECT Book_Name
FROM Books_Details 
WHERE Book_Category_Code=(SELECT Book_Category_code FROM Categories WHERE Categories="Music");"""
cursor.execute(query)
result=cursor.fetchall()
print(result)

[('Rip it Up and Start Again',), ('Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991',), ('How Music Works',), ('Love Is a Mix Tape (Music #1)',), ('Please Kill Me: The Uncensored Oral History of Punk',), ("Kill 'Em and Leave: Searching for James Brown and the American Soul",), ('This Is Your Brain on Music: The Science of a Human Obsession',), ('Orchestra of Exiles: The Story of Bronislaw Huberman, the Israel Philharmonic, and the One Thousand Jews He Saved from Nazi Horrors',), ('No One Here Gets Out Alive',), ('Life',), ('Forever Rockers (The Rocker #12)',)]
